# Using Slurm

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
os.chdir('../..')

In [2]:
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    project="takco",
    cores=8,
    memory="50 GB",
    scheduler_options={"dashboard_address": ":37659"},
)
cluster.scale(jobs=10)
cluster

SLURMCluster('tcp://130.37.199.8:41539', workers=0, threads=0, memory=0 B)

In [ ]:
import logging
logging.getLogger().setLevel(logging.INFO)

import takco
pipe = takco.Config('resources/pipelines/TabEL.toml')
pipe['step'][1]['split_compound_columns'] = False

tables = takco.TableSet.run(pipe, **pipe, executor={'class':'DaskHashBag', 'address': cluster})
tables = sorted(tables, key=lambda x: -x['numDataRows'])

INFO:root:Running pipeline 'TabEL' in None
INFO:root:Chaining pipeline step 0-load: {'step': 'load', 'vals': 'data/TabEL/part-*', 'tables': [], 'workdir': '.', 'datadir': 'data', 'resourcedir': None, 'assets': [{'name': 'tabel', 'class': 'Annotation', 'download': ['http://websail-fe.cs.northwestern.edu/TabEL/tables.json.gz'], 'path': 'data/TabEL', 'fname': 'tables.json.gz'}], 'kbs': None, 'executors': None, 'cache': None, 'executor': {'class': 'DaskHashBag', 'address': SLURMCluster('tcp://130.37.199.8:41539', workers=40, threads=80, memory=500.00 GB)}}
INFO:root:Chaining pipeline step 1-reshape: {'step': 'reshape', 'split_compound_columns': False, 'heuristics': [{'class': 'NumSuffix'}], 'tables': <takco.util.DaskHashBag object at 0x2aaabf2ddbb0>, 'workdir': '.', 'datadir': 'data', 'resourcedir': None, 'assets': [{'name': 'tabel', 'class': 'Annotation', 'download': ['http://websail-fe.cs.northwestern.edu/TabEL/tables.json.gz'], 'path': 'data/TabEL', 'fname': 'tables.json.gz'}], 'kbs': N

In [10]:
len(tables)

4916

In [11]:
takco.preview(tables, nrows=10)

?,0,1,2,3
,_pgTitle,Year,First consul,Second consul
,List of Roman consuls,509,L. Iunius Brutus,L. Tarquinius Collatinus
,List of Roman consuls,suff.,Sp. Lucretius Tricipitinus,P. Valerius Poplicola
,List of Roman consuls,suff.,M. Horatius Pulvillus,
,List of Roman consuls,508,P. Valerius Poplicola II,T. Lucretius Tricipitinus
,List of Roman consuls,507,P. Valerius Poplicola III,M. Horatius Pulvillus II
,List of Roman consuls,506,Sp. Larcius Rufus (or Flavus),T. Herminius Aquilinus
,List of Roman consuls,505,M. Valerius Volusus,P. Postumius Tubertus
,List of Roman consuls,504,P. Valerius Poplicola IV,T. Lucretius Tricipitinus II
,List of Roman consuls,503,Agrippa Menenius Lanatus,P. Postumius Tubertus II
